# 01 - Data preprocessing and feature engineering

## Prepare Environment

In [ ]:
!pip install accelerate>=0.20.1
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yhkqi_jn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yhkqi_jn
  Resolved https://github.com/huggingface/transformers to commit 0ea42ef0f9f71deba7775ead33afa0e493823d60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Huggingface login
We need it to download the dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We will use Google Drive as a feature store

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

DATASET_RAW = '/gdrive/My Drive/scalable/dataset_raw/'
DATASET = '/gdrive/My Drive/scalable/dataset/'
DATASET_MAP = '/gdrive/My Drive/scalable/dataset_mapped/'

Mounted at /gdrive


We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files.

## Load Dataset

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import load_from_disk
import os

common_voice = DatasetDict()

if not os.path.exists(DATASET_RAW):
  common_voice["train_0"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train[:2500]", use_auth_token=True, keep_in_memory=True)
  common_voice["train_1"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train[2500:5000]", use_auth_token=True, keep_in_memory=True)
  common_voice["train_2"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train[5000:7500]", use_auth_token=True, keep_in_memory=True)
  common_voice["train_3"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train[7500:10000]", use_auth_token=True, keep_in_memory=True)
  common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="test[:2500]", use_auth_token=True, keep_in_memory=True)

  os.mkdir(DATASET_RAW)
  print(common_voice)

  common_voice.save_to_disk(DATASET_RAW)

  print('Dataset saved!')

else:
  common_voice = load_from_disk(DATASET_RAW, keep_in_memory=False)


Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train_0: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2500
    })
    train_1: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2500
    })
    train_2: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2500
    })
    train_3: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2500
    })
})


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:
1) A feature extractor which pre-processes the raw audio-inputs
2) The model which performs the sequence-to-sequence mapping
3) A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Hindi, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

### Prepare Data

Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [1]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import load_from_disk
import os

if not os.path.exists(DATASET_MAP):
  common_voice['train_0'] = common_voice['train_0'].map(prepare_dataset, num_proc=1, keep_in_memory=False)
  common_voice['train_1'] = common_voice['train_1'].map(prepare_dataset, num_proc=1)
  common_voice['train_2'] = common_voice['train_2'].map(prepare_dataset, num_proc=1)
  common_voice['train_3'] = common_voice['train_3'].map(prepare_dataset, num_proc=1)
  common_voice['test'] = common_voice['test'].map(prepare_dataset, num_proc=1)

  os.mkdir(DATASET_MAP)
  common_voice.save_to_disk(DATASET_MAP)
else:
  common_voice = load_from_disk(DATASET_MAP, keep_in_memory=False)


In [ ]:
from datasets import concatenate_datasets
train_dataset = concatenate_datasets([common_voice['train_0'], common_voice['train_1'], common_voice['train_2'], common_voice['train_3']])

In [ ]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'input_features', 'labels'],
    num_rows: 10000
})